In [1]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 39.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.4 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
albucore 0.0.23 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [2]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Tue Feb 25 16:12:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8       

In [3]:
# TIF Image Denoising Script for Google Colab
# This script processes TIF files from folders in Google Drive

import os
import json
import time
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import tifffile
from cellpose import denoise, io
import numpy as np
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive - Run this first to access your files
drive.mount('/content/drive')

# Setup paths - Change these if needed
INPUT_BASE_PATH = '/content/drive/MyDrive/knowledge/University/Master/Thesis/TIF_Converted'
OUTPUT_BASE_PATH = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised'
LOG_FILE = '/content/drive/MyDrive/knowledge/University/Master/Thesis/processed_folders.json'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_BASE_PATH, exist_ok=True)

# Setup cellpose logging
io.logger_setup()

# Define channel names in order
channel_names = ['VE-Cadherin', 'DAPI', 'Golgi']

# Create models for denoising with appropriate types
channel_models = {
    'VE-Cadherin': denoise.CellposeDenoiseModel(gpu=True, model_type="cyto3", restore_type="denoise_cyto3"),
    'DAPI': denoise.CellposeDenoiseModel(gpu=True, model_type="nuclei", restore_type="denoise_cyto3"),
    'Golgi': denoise.CellposeDenoiseModel(gpu=True, model_type="cyto3", restore_type="denoise_cyto3")
}

# Set typical diameters for each structure
diameters = {
    'VE-Cadherin': 35,  # Cell membrane structures
    'DAPI': 30,         # Nuclei are typically smaller
    'Golgi': 30         # Golgi structures
}

def load_processed_folders():
    """Load the list of already processed folders"""
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'r') as f:
            return json.load(f)
    return []

def save_processed_folder(folder_name):
    """Mark a folder as processed"""
    processed = load_processed_folders()
    if folder_name not in processed:
        processed.append(folder_name)
        with open(LOG_FILE, 'w') as f:
            json.dump(processed, f)

def process_tif_file(file_path, output_folder):
    """Process a single TIF file"""
    print(f"Processing file: {os.path.basename(file_path)}")

    # Read the TIFF file
    imgs = tifffile.imread(file_path)

    # Process each channel separately for all Z slices
    denoised_images = []

    # Main processing loop
    for channel, name in enumerate(channel_names):
        if channel >= imgs.shape[1]:
            print(f"Warning: Channel {channel} ({name}) not found")
            continue

        print(f"Processing {name} channel...")
        channel_denoised = []

        for z in tqdm(range(imgs.shape[0])):
            img = imgs[z, channel].astype(np.float32)

            # Normalize image to 0-1 range
            min_val = img.min()
            max_val = img.max()
            if max_val > min_val:
                img = (img - min_val) / (max_val - min_val)
            img = img[np.newaxis, ...]

            # Run denoising
            _, _, _, img_denoised = channel_models[name].eval(
                img,
                diameter=diameters[name],
                channels=[0,0]
            )

            channel_denoised.append(img_denoised.squeeze())

        if channel_denoised:
            denoised_images.append(np.stack(channel_denoised))

    # Save the denoised images
    if denoised_images:
        output_stack = np.stack(denoised_images)
        output_file = os.path.join(output_folder, f"denoised_{os.path.basename(file_path)}")
        tifffile.imwrite(output_file, output_stack)
        print(f"Saved to: {output_file}")
        return True
    else:
        print("No denoised images produced.")
        return False

def process_folder(folder_name):
    """Process all TIF files in a folder"""
    input_folder = os.path.join(INPUT_BASE_PATH, folder_name)
    output_folder = os.path.join(OUTPUT_BASE_PATH, folder_name)

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    print(f"\n=== Processing folder: {folder_name} ===")

    # Get all TIF files in the folder
    tif_files = [f for f in os.listdir(input_folder)
                if f.lower().endswith('.tif') or f.lower().endswith('.tiff')]

    if not tif_files:
        print(f"No TIF files found in {input_folder}")
        return False

    # Process each TIF file
    files_processed = 0
    for tif_file in tif_files:
        try:
            success = process_tif_file(os.path.join(input_folder, tif_file), output_folder)
            if success:
                files_processed += 1
        except Exception as e:
            print(f"Error processing {tif_file}: {e}")

    print(f"Completed folder {folder_name}: {files_processed}/{len(tif_files)} files processed")
    return files_processed > 0

# Get list of folders
def get_folders():
    """Get all folders in the input directory"""
    return [f for f in os.listdir(INPUT_BASE_PATH)
           if os.path.isdir(os.path.join(INPUT_BASE_PATH, f))]

# Function to process a specific folder
def process_specific_folder(folder_name):
    """Process a single specified folder"""
    if folder_name not in get_folders():
        print(f"Folder {folder_name} not found in {INPUT_BASE_PATH}")
        return

    processed = load_processed_folders()
    if folder_name in processed:
        print(f"Folder {folder_name} already processed. Processing again...")

    success = process_folder(folder_name)
    if success:
        save_processed_folder(folder_name)
        print(f"Folder {folder_name} completed and saved to log.")
    else:
        print(f"Folder {folder_name} processing failed or had no files.")

# Display available folders for selection
def display_folders():
    """Show available folders and their processing status"""
    all_folders = get_folders()
    processed = load_processed_folders()

    print("\nAvailable folders:")
    print("------------------")
    for i, folder in enumerate(all_folders):
        status = "✓ Processed" if folder in processed else "□ Not processed"
        print(f"{i+1}. {folder} - {status}")

    return all_folders

# Optional: Show details on GPU usage
def show_gpu_info():
    """Display information about the available GPU"""
    try:
        !nvidia-smi
    except:
        print("No GPU information available")

# Function to process all remaining folders
def process_all_remaining():
    """Process all folders that haven't been processed yet"""
    all_folders = get_folders()
    processed = load_processed_folders()

    folders_to_process = [f for f in all_folders if f not in processed]

    if not folders_to_process:
        print("All folders have been processed!")
        return

    print(f"Will process {len(folders_to_process)} remaining folders")

    for folder in folders_to_process:
        success = process_folder(folder)
        if success:
            save_processed_folder(folder)
            print(f"Folder {folder} completed and saved to log.")
        else:
            print(f"Folder {folder} processing failed or had no files.")

# Show processing time estimate
def estimate_processing_time(num_folders=1):
    """Provide a rough estimate of processing time"""
    # Rough estimate based on typical processing
    per_file_time = 2  # minutes per file
    files_per_folder = 5  # estimated files per folder

    total_time = num_folders * files_per_folder * per_file_time

    print(f"Estimated processing time: ~{total_time} minutes")
    print("Note: Actual time may vary based on file sizes and GPU performance")

# Main menu for interactive usage
def show_menu():
    """Display interactive menu for folder selection"""
    print("\n=== TIF Image Denoising Tool ===")
    print("1. Display available folders")
    print("2. Process a specific folder")
    print("3. Process all remaining folders")
    print("4. Show GPU information")
    print("5. Exit")

    choice = input("\nEnter your choice (1-5): ")

    if choice == '1':
        display_folders()
    elif choice == '2':
        all_folders = display_folders()
        folder_idx = input("\nEnter folder number to process: ")
        try:
            folder_idx = int(folder_idx) - 1
            if 0 <= folder_idx < len(all_folders):
                estimate_processing_time(1)
                process_specific_folder(all_folders[folder_idx])
            else:
                print("Invalid folder number")
        except ValueError:
            print("Please enter a number")
    elif choice == '3':
        all_folders = get_folders()
        processed = load_processed_folders()
        remaining = len([f for f in all_folders if f not in processed])

        estimate_processing_time(remaining)
        confirm = input(f"Process all {remaining} remaining folders? (y/n): ")
        if confirm.lower() == 'y':
            process_all_remaining()
    elif choice == '4':
        show_gpu_info()
    elif choice == '5':
        print("Exiting program")
        return False
    else:
        print("Invalid choice")

    return True

# Start interactive menu
# This will keep showing the menu until user chooses to exit
def run_interactive():
    """Run the interactive menu"""
    continue_running = True
    while continue_running:
        continue_running = show_menu()

# To start the interactive menu, uncomment the line below:
run_interactive()

Mounted at /content/drive
creating new log file


100%|██████████| 25.3M/25.3M [00:02<00:00, 9.26MB/s]
100%|██████████| 25.3M/25.3M [00:02<00:00, 9.16MB/s]
100%|██████████| 25.3M/25.3M [00:02<00:00, 9.24MB/s]



=== TIF Image Denoising Tool ===
1. Display available folders
2. Process a specific folder
3. Process all remaining folders
4. Show GPU information
5. Exit

Enter your choice (1-5): 1

Available folders:
------------------
1. Static-A-2 - ✓ Processed
2. flow3_1.4Pa_18h - ✓ Processed
3. flow5_8pa_5h - ✓ Processed
4. 1.4Pa-A-2_40x - □ Not processed
5. Static-A-1 - □ Not processed
6. 1.4Pa-A-2 - □ Not processed
7. 1.4Pa-A-1 - □ Not processed

=== TIF Image Denoising Tool ===
1. Display available folders
2. Process a specific folder
3. Process all remaining folders
4. Show GPU information
5. Exit

Enter your choice (1-5): 3
Estimated processing time: ~40 minutes
Note: Actual time may vary based on file sizes and GPU performance
Process all 4 remaining folders? (y/n): y
Will process 4 remaining folders

=== Processing folder: 1.4Pa-A-2_40x ===
Processing file: 1.4Pa_A1_20dec21_40x_L2RA_FlatA_seq001.tif
Processing VE-Cadherin channel...


  0%|          | 0/21 [00:00<?, ?it/s]

Processing DAPI channel...


...

Processing DAPI channel...


  0%|          | 0/13 [00:00<?, ?it/s]

Processing Golgi channel...


  0%|          | 0/13 [00:00<?, ?it/s]

Saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/1.4Pa-A-1/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq009.tif
Completed folder 1.4Pa-A-1: 22/22 files processed
Folder 1.4Pa-A-1 completed and saved to log.

=== TIF Image Denoising Tool ===
1. Display available folders
2. Process a specific folder
3. Process all remaining folders
4. Show GPU information
5. Exit

Enter your choice (1-5): 1

Available folders:
------------------
1. Static-A-2 - ✓ Processed
2. flow3_1.4Pa_18h - ✓ Processed
3. flow5_8pa_5h - ✓ Processed
4. 1.4Pa-A-2_40x - ✓ Processed
5. Static-A-1 - ✓ Processed
6. 1.4Pa-A-2 - ✓ Processed
7. 1.4Pa-A-1 - ✓ Processed

=== TIF Image Denoising Tool ===
1. Display available folders
2. Process a specific folder
3. Process all remaining folders
4. Show GPU information
5. Exit

Enter your choice (1-5): 5
Exiting program
